# Why

Rasterix provides a `RasterIndex` that uses an affine transform to enable indexing of the underlying data. 

Why should you use it?
1. It eliminates an entire class of bugs where Xarray allows you to add (for example) two datasets with different affine transforms (and/or projections) and return nonsensical outputs.
2. It enables indexing using the coordinate transformation, minimizing impacts of any floating-point mismatches.
3. To fit the Xarray data model, RasterIndex creates lazy coordinate variables and propagated them during indexing as much as possible. Thus very large coordinates can be represented with minimal memory cost.

## Quick demo

Below we quickly demonstrate some of these features. Note that this particular example notebook skips CRS handling.

In [ ]:
%xmode minimal

import numpy as np
import xarray as xr

import rasterix

np.set_printoptions(threshold=10, edgeitems=2)
xr.set_options(display_expand_indexes=True)

We will demonstrate the raster index with a simple dataset with  rectilinear coordinates and no rotation or skew.
Both x and y coordinates are 1-dimensional.

In [ ]:
source = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"

## With and without `RasterIndex`.

In [ ]:
da_no_raster_index = xr.open_dataarray(source, engine="rasterio")
da_no_raster_index

In [ ]:
da_raster_index = rasterix.assign_index(da_no_raster_index)
da_raster_index

Let's compare the coordinates for the DataArrays with and without RasterIndex.

Without a raster index, `x` and `y` are in-memory data.

In [ ]:
da_no_raster_index.x

By contrast, with a RasterIndex, coordinate values are lazy and generated on-demand!

In [ ]:
da_raster_index.x

These differences are viewable in the repr. Note the `PandasIndex` type under "Indexes".

In [ ]:
da_no_raster_index

 The repr below shows a few values for each coordinate (those have been computed on-the-fly) but clicking on the database icon doesn't show any value in the spatial coordinate data reprs.

In [ ]:
da_raster_index

The generated coordinate values correspond to cell _centers_.

In [ ]:
da_raster_index.x.to_numpy()

The affine transforms are accessible:

In [ ]:
da_raster_index.xindexes["x"].transform()  # top-left

In [ ]:
da_raster_index.xindexes["x"].center_transform()  # pixel centrres

## Equality

`equals` compares variable values without relying on Xarray coordinate indexes. Both dataarrays should thus be equal.

In [ ]:
da_raster_index.equals(da_no_raster_index)

## Alignment

Xarray alignment relies on Xarray coordinate indexes. Trying to align both datasets fails here since they each have different index types.

In [ ]:
da_raster_index + da_no_raster_index